In [ ]:
import os
! apt-get update -qq > /dev/null   
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
!pip install nlu  pyspark==2.4.7 > /dev/null   
!pip install flask_ngrok
import nlu

In [ ]:
API = "http://ec2-65-2-29-164.ap-south-1.compute.amazonaws.com:5000/api/"

In [ ]:
from flask import Flask,request,jsonify
from flask_ngrok import run_with_ngrok
from sklearn.metrics.pairwise import cosine_similarity

import requests

app = Flask(__name__)
run_with_ngrok(app)

multi_pipe = None
col_names = ['sentence_embedding_bert','sentence_embedding_electra', 'sentence_embedding_use']

@app.route("/load-api")
def loadAPI():
    try:
        global multi_pipe
        multi_pipe = nlu.load('use en.embed_sentence.electra embed_sentence.bert')
        return {"status" : 200},200
    except:
        return {"status" : 500},500

@app.route("/generate-predictions",methods = ['GET', 'POST'])
def genearatePredictions():

    title = request.form['title']
    
    if title and multi_pipe:
        predictions = multi_pipe.predict(title,get_embeddings=True).iloc[0]
        
        response = {}

        for col_name in col_names:
            response[col_name] = list(predictions[col_name][0])
        
        return { "predictions" : response ,"status" : 201},201
    else:
        return { "error" :"ERROR" ,"status" : 500}, 500

@app.route("/get-similar-questions",methods=['GET','POST'])
def getSimilarQuestions():
    data = requests.get(API+"discussion/predictions").json()
    embeddings = request.form['predictions']

    if data['status'] == 200 and embeddings:
        predictions = data['predictions']
        for i in range(len(predictions)):
            similarity = 0
            for e_col in col_names:
                if e_col in predictions[i] and e_col in embeddings:
                    similarity += cosine_similarity(predictions[i][e_col],embeddings[e_col]) 

            predictions[i]['similarity_score'] = similarity/len(col_names)
        
        return {"similarQuestions" : sorted(predictions,key = lambda x : x['similarity_score'],reverse=True)[:3] , "status" : 201} , 201
    else:
        return { "status" : 500,"error" :"ERROR"}, 500

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7d28-34-86-229-159.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [26/Nov/2021 16:51:23] "POST /generate-predictions HTTP/1.1" 500 -
127.0.0.1 - - [26/Nov/2021 16:51:24] "POST /get-similar-questions HTTP/1.1" 201 -


[{'predictions': {'sentence_embedding_bert': [], 'sentence_embedding_electra': [], 'sentence_embedding_use': []}, '_id': '61a1089796b029268c26db06'}]


...

127.0.0.1 - - [26/Nov/2021 16:56:34] "GET /load-api HTTP/1.1" 200 -



tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sent_electra_small_uncased download started this may take some time.
Approximate size to download 48.7 MB
[OK!]
sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]

127.0.0.1 - - [26/Nov/2021 16:56:52] "GET /load-api HTTP/1.1" 200 -


127.0.0.1 - - [26/Nov/2021 16:56:59] "POST /generate-predictions HTTP/1.1" 201 -
127.0.0.1 - - [26/Nov/2021 16:57:01] "POST /get-similar-questions HTTP/1.1" 201 -


[{'predictions': {'sentence_embedding_bert': [], 'sentence_embedding_electra': [], 'sentence_embedding_use': []}, '_id': '61a1089796b029268c26db06'}, {'predictions': {'sentence_embedding_bert': [], 'sentence_embedding_electra': [], 'sentence_embedding_use': []}, '_id': '61a1108caa807637d4cc7450'}, {'predictions': {'sentence_embedding_bert': [], 'sentence_embedding_electra': [], 'sentence_embedding_use': []}, '_id': '61a1110ce421e512b8a29c9f'}, {'predictions': {'sentence_embedding_bert': [], 'sentence_embedding_electra': [], 'sentence_embedding_use': []}, '_id': '61a11147444f8d311c033a60'}, {'predictions': {'sentence_embedding_bert': [], 'sentence_embedding_electra': [], 'sentence_embedding_use': []}, '_id': '61a1118296b24f160015b1a8'}, {'predictions': {'sentence_embedding_bert': [], 'sentence_embedding_electra': [], 'sentence_embedding_use': []}, '_id': '61a11194af276e39ec6cc194'}]
